Supervised Learning - Classification and Regression

# In Depth: Decision Trees and Random Forests

## Motovating Random Forests: Decision Trees

## Ensembles of Estimators: Random Forests

## Random Forest Regression

## Example: Random Forest for Classifying Digits

## Summary of Random Forest

This section contained a brief introduction to the concept of **ensemble estimators**, and in particular the random forest model - an ensemble of randomized decision trees. Random forests are a powerful method with serveral advantages:

- Both training and prediction are very fast, because of the simplicity of the underlying decision trees. In addition, both tasks can be straightforwardly parallelized, because the individual trees are entirely independent entities.
- The multiple trees allow for a probabilistic classification: a majority vote among estimators gives an estimate of the probability (accessed in Scikit-Learn with the `predict_proba()` method).
- The nonparametric model is extremely flexible, and can thus perform well on tasks that are underfit by other estimators.

A primary disadvantage of random forests is that the results are not easily interpretable; that is, if you would like to draw conclusions about the **meaning** of the classification model, random forests may not be the best choice.